In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = (high/low)**2
    
    
    
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, pbias3, pred_day, labels, mae, rmse, R, (high/low)

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:

run_num = range(len(folder))

#run_num = [0, 1]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2013.xlsx', '가평_2014.xlsx', '가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2013.xlsx', '서상_2014.xlsx', '서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2013.xlsx', '의암호_2014.xlsx', '의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2013.xlsx
data/han/자동/가평_2014.xlsx
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/서상_2013.xlsx
data/han/자동/서상_2014.xlsx
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  201

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5878 - disc_loss: 0.3672 - rmse: 0.4462 - val_loss: 0.4713
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.9958 - disc_loss: 0.3688 - rmse: 0.3775 - val_loss: 0.3884
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.2630 - disc_loss: 0.3448 - rmse: 0.4241 - val_loss: 0.4550
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.2270 - disc_loss: 0.3859 - rmse: 0.5023 - val_loss: 0.4317
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7063 - disc_loss: 0.3383 - rmse: 0.3792 - val_loss: 0.5016
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.3189 - disc_loss: 0.3825 - rmse: 0.4028 - val_loss: 0.4114
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.9695 - disc_loss: 0.3784 - rmse: 0.4265 - val_loss: 0.3999
Epoch 56/2000
1/1 [================

1/1 [==============================] - 0s 20ms/step - gen_loss: 24.3957 - disc_loss: 0.2960 - rmse: 0.4817 - val_loss: 0.3852
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3051 - disc_loss: 0.3083 - rmse: 0.3661 - val_loss: 0.3621
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5081 - disc_loss: 0.2995 - rmse: 0.3547 - val_loss: 0.3882
Epoch 110/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.5763 - disc_loss: 0.2964 - rmse: 0.3956 - val_loss: 0.4491
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4241 - disc_loss: 0.3056 - rmse: 0.3563 - val_loss: 0.3868
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0344 - disc_loss: 0.3060 - rmse: 0.3462 - val_loss: 0.4268
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5364 - disc_loss: 0.3174 - rmse: 0.4289 - val_loss: 0.4408
Epoch 114/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.3944 - disc_loss: 0.2826 - rmse: 0.5064 - val_loss: 0.3656
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0164 - disc_loss: 0.2519 - rmse: 0.3595 - val_loss: 0.3480
Epoch 167/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.0610 - disc_loss: 0.2597 - rmse: 0.4473 - val_loss: 0.3929
Epoch 168/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.3608 - disc_loss: 0.2596 - rmse: 0.4235 - val_loss: 0.3189
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9612 - disc_loss: 0.2631 - rmse: 0.3558 - val_loss: 0.4015
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4808 - disc_loss: 0.2532 - rmse: 0.3222 - val_loss: 0.3400
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.0881 - disc_loss: 0.2550 - rmse: 0.3761 - val_loss: 0.3608
Epoch 172/2000
1/1 [===========

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1459 - disc_loss: 0.2543 - rmse: 0.3464 - val_loss: 0.3779
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4802 - disc_loss: 0.2433 - rmse: 0.3482 - val_loss: 0.3325
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0899 - disc_loss: 0.2575 - rmse: 0.3283 - val_loss: 0.3338
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.1737 - disc_loss: 0.2499 - rmse: 0.4086 - val_loss: 0.3472
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6674 - disc_loss: 0.2647 - rmse: 0.3323 - val_loss: 0.3459
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0229 - disc_loss: 0.2445 - rmse: 0.3390 - val_loss: 0.4385
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1346 - disc_loss: 0.2560 - rmse: 0.3898 - val_loss: 0.3860
Epoch 230/2000
1/1 [==============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7491 - disc_loss: 0.1906 - rmse: 0.3191 - val_loss: 0.3110
Epoch 746/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1574 - disc_loss: 0.1964 - rmse: 0.2774 - val_loss: 0.3221
Epoch 747/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 17.7833 - disc_loss: 0.1834 - rmse: 0.3309 - val_loss: 0.5909
Epoch 748/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1683 - disc_loss: 0.1866 - rmse: 0.3601 - val_loss: 0.3761
Epoch 749/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1219 - disc_loss: 0.1914 - rmse: 0.2946 - val_loss: 0.3299
Epoch 750/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.7585 - disc_loss: 0.1806 - rmse: 0.2752 - val_loss: 0.3402
Epoch 751/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5653 - disc_loss: 0.1910 - rmse: 0.3327 - val_loss: 0.2956
Epoch 752/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5156 - disc_loss: 0.1921 - rmse: 0.3033 - val_loss: 0.3276
Epoch 804/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9368 - disc_loss: 0.1897 - rmse: 0.3608 - val_loss: 0.3334
Epoch 805/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4218 - disc_loss: 0.1920 - rmse: 0.4053 - val_loss: 0.3745
Epoch 806/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5390 - disc_loss: 0.1907 - rmse: 0.3620 - val_loss: 0.3628
Epoch 807/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9579 - disc_loss: 0.1887 - rmse: 0.3101 - val_loss: 0.3965
Epoch 808/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0683 - disc_loss: 0.1875 - rmse: 0.3398 - val_loss: 0.2585
Epoch 809/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8740 - disc_loss: 0.1851 - rmse: 0.2781 - val_loss: 0.2553
Epoch 810/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3166 - disc_loss: 0.1690 - rmse: 0.3830 - val_loss: 0.3282
Epoch 862/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3766 - disc_loss: 0.1826 - rmse: 0.3827 - val_loss: 0.3323
Epoch 863/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6951 - disc_loss: 0.1860 - rmse: 0.3376 - val_loss: 0.3210
Epoch 864/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1551 - disc_loss: 0.1914 - rmse: 0.3734 - val_loss: 0.2783
Epoch 865/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0501 - disc_loss: 0.1876 - rmse: 0.3875 - val_loss: 0.2798
Epoch 866/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7563 - disc_loss: 0.1917 - rmse: 0.3127 - val_loss: 0.2754
Epoch 867/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0401 - disc_loss: 0.1713 - rmse: 0.3706 - val_loss: 0.3715
Epoch 868/2000
1/1 [===============

1/1 [==============================] - 0s 409ms/step - gen_loss: 4.7224 - disc_loss: 0.1857 - rmse: 0.2798 - val_loss: 0.2952
Epoch 920/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1424 - disc_loss: 0.1835 - rmse: 0.2946 - val_loss: 0.3928
Epoch 921/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1575 - disc_loss: 0.1727 - rmse: 0.3176 - val_loss: 0.3615
Epoch 922/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7703 - disc_loss: 0.1721 - rmse: 0.3117 - val_loss: 0.3320
Epoch 923/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.9864 - disc_loss: 0.1746 - rmse: 0.4265 - val_loss: 0.3702
Epoch 924/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.6829 - disc_loss: 0.1863 - rmse: 0.4544 - val_loss: 0.3345
Epoch 925/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0945 - disc_loss: 0.1654 - rmse: 0.3250 - val_loss: 0.3039
Epoch 926/2000
1/1 [============

data/han/총량/청평_2013.xlsx
data/han/총량/청평_2014.xlsx
data/han/총량/청평_2015.xlsx
data/han/총량/청평_2016.xlsx
data/han/총량/청평_2017.xlsx
data/han/총량/청평_2018.xlsx
data/han/총량/청평_2019.xlsx
data/han/총량/청평_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/총량/가평천3_2013.xlsx
data/han/총량/가평천3_2014.xlsx
data/han/총량/가평천3_2015.xlsx
data/han/총량/가평천3_2016.xlsx
data/han/총량/가평천3_2017.xlsx
data/han/총량/가평천3_2018.xlsx
data/han/총량/가평천3_2019.xlsx
data/han/총량/가평천3_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/총량/춘성교_2013.xlsx
data/han/총량/춘성교_2014.xlsx
data/han/총량/춘성교_2015.xlsx
data/han/총량/춘성교_2016.xlsx
data/han/총량/춘성교_2017.xlsx
data/han/총량/춘성교_2018.xlsx
data/han/총량/춘성교_2019.xlsx
data/han/총량/춘성교_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/총량/화천_2013.xlsx
data/han/총량/화천_2014.xlsx
data/han/총량/화천_2015.xlsx
data/han/총량/화천_2016.xlsx
data/han/총량/화천_2017.xlsx
data/han/총량/화천_2018.xlsx
data/han/총량/화천_2019.xlsx
data/han/총량/화천_2020.x

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [40]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(66456, 181)
toc


In [41]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (46519, 181) val_df.shape :  (13292, 181) test_df.shape: (6645, 181)


In [42]:
#fake_df.shape[0]/8760

In [43]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [44]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [45]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [46]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han/toc/


In [49]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [50]:
rnn_epochs, rnn_in_setps

(20, 240)

In [51]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 3

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    nse, _, _, _ = compa(
        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -7):
        b = 3
    if(bob > 7):
        b = -3
'''
    


'\nsub_flag = True\nhistory_nse = []\n\nbob = 0\nb = 3\n\nwhile sub_flag:\n    \n\n    \n    gru_model = model_gru(\n        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,\n        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,\n        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)\n    nse, _, _, _ = compa(\n        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n        #predict_day = rnn_predict_day)\n        predict_day = 4)\n    \n    if(nse > 0.5):\n        sub_flag = False\n   \n    \n    history_nse.append(nse)\n    \n    print(\'--------------------------\')\n    print(history_nse)\n    print(nse)\n    print(\'--------------------------\')\n    \n        \n    bob += b\n    if(bob < -7):\n        b = 3\n    if(bob > 7):\n        b = -3\n'

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 10,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
10/10 [==============================] - 46s 5s/step - loss: 0.6803 - mean_absolute_error: 0.5812 - nse: -0.0286 - val_loss: 0.4713 - val_mean_absolute_error: 0.5725 - val_nse: -0.2364
Epoch 2/15
10/10 [==============================] - 45s 5s/step - loss: 0.5320 - mean_absolute_error: 0.4580 - nse: 0.3664 - val_loss: 0.3673 - val_mean_absolute_error: 0.5016 - val_nse: 0.0370
Epoch 3/15
10/10 [==============================] - 42s 4s/step - loss: 0.3704 - mean_absolute_error: 0.4039 - nse: 0.4669 - val_loss: 0.3830 - val_mean_absolute_error: 0.5103 - val_nse: -0.0060
Epoch 4/15
10/10 [==============================] - ETA: 0s - loss: 0.4991 - mean_absolute_error: 0.3853 - nse: 0.4450

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)

print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _R)

#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _R)



In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _R)

#print('nse_train : ', val_nse['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _R)

In [34]:
#train_df.shape, val_df.shape, test_df.shape

In [35]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
